In [1]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [2]:
import tensorflow as tf
import numpy as np


print(tf.test.is_gpu_available())

False


In [3]:
layer = tf.keras.layers.Dense(100)
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))



In [4]:
layer(tf.zeros([10, 5]))


<tf.Tensor: id=29, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.3667961 , -0.284721  ,  0.19175869,  0.05165207,  0.20846522,
          0.49332768,  0.26590765, -0.10697991,  0.31543916,  0.3061149 ],
        [-0.5074483 ,  0.22887784, -0.19464949, -0.38417488, -0.43215704,
         -0.36336958, -0.4751143 ,  0.3541615 ,  0.40829986,  0.07605004],
        [-0.49695912, -0.47461563, -0.62234646, -0.6167852 ,  0.57956606,
          0.4206385 ,  0.3128522 , -0.30039415,  0.3609562 ,  0.19601095],
        [ 0.5919307 ,  0.24778819,  0.22133452,  0.3329311 , -0.19983059,
          0.23575413,  0.07985449, -0.47218263,  0.6106983 , -0.13562128],
        [ 0.07527995, -0.49805686, -0.15046903, -0.5429091 , -0.18559143,
         -0.29944205, -0.00996614, -0.5413599 ,  0.02887869, -0.4918344 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [6]:

class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)



In [7]:
layer = MyDenseLayer(10)
_ = layer(tf.zeros([10, 5]))



In [8]:
print([var.name for var in layer.trainable_variables])


['my_dense_layer_1/kernel:0']


In [9]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)




In [10]:
block = ResnetIdentityBlock(1, [1, 2, 3])

In [11]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [12]:
block.layers

In [13]:
len(block.variables)

18

In [14]:
block.summary()

Model: "resnet_identity_block_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_3 (Batch multiple                  4         
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_4 (Batch multiple                  8         
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_5 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
______________________________________________

In [15]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])


In [16]:
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=738, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [17]:
my_seq.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_6 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_7 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_8 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________